### Getting data from the Google Earth Engine

In [23]:
# importing packages and initializing Google Earth Engine
%matplotlib inline
import matplotlib.pyplot as plt
import ee
import urllib
import zipfile
import os
import numpy as np

ee.Initialize()

In [8]:
folder_loc = '/Users/patrickdoupe/Dropbox/insightDataScience/project/'
year = '2010'
if not os.path.exists(folder_loc+'LANDSAT_TOA/small'):
    os.makedirs(folder_loc+'LANDSAT_TOA/small')

In [9]:
## Getting Satellite data for Oregon
# Load LANDSAT TOA image for 2010
image = ee.Image('LANDSAT/LE7_TOA_1YEAR/'+year) 



In [40]:
lon = np.arange(-124, -120, 0.5)
lat = np.arange(43, 44, 0.5)
## Now export the scenes from the ImageCollection to 
## Google Drive
for i in range(len(lon)):
    for j in range(len(lat)):
        print lat[j], lon[i]
        print lat[j+1], lon[i+1]
        rectangle = ee.Geometry.Rectangle(lat[j], lon[i], lat[j+1], lon[i+1])
        feature = ee.Feature(rectangle)
        image = image.clip(feature)
        im_download = image.select(['B1'])
        path = image.getDownloadURL(
          {'scale': 30, 
          'crs': 'EPSG:2913',
          'maxPixels' : 1e13
          }
        );
        spatial_data_file = 'B1_240m_epsg_2913_' + str(lat[j]) + '_' + str(lon[i])
        a = urllib.urlretrieve(path, spatial_data_file)
        zfile = zipfile.ZipFile(spatial_data_file)
        zfile.extractall('LANDSAT_TOA/small')

43.0 -124.0
43.5 -123.5


EEException: The requested image band is too large: was 8495270 bytes per scanline; must be no more than 262144 bytes per scanline.

In [95]:
state_name = 'Oregon'
year = '2010'


if not os.path.exists(folder_loc+'LANDSAT_TOA/'+state_name+'/small'):
    os.makedirs(folder_loc+'LANDSAT_TOA/'+state_name+'/small')

In [96]:
## Getting Satellite data for Oregon
# Load LANDSAT TOA image for 2010
image = ee.Image('LANDSAT/LE7_TOA_1YEAR/'+year) 

# Get State boundaries
featureCollection = ee.FeatureCollection('ft:1fRY18cjsHzDgGiJiS2nnpUU3v9JPDc2HNaR7Xk8');
# Get a map of the state
state_filter = featureCollection.filter(ee.Filter.eq('Name', state_name));
border = ee.Feature(state_filter.first())

# Make a bitmask that we can use to set the values outside the
# region to zero (for download), and then also call clip() to 
# set the mask (for things like addToMap() that use the mask)
outside = ee.Image(1).uint8().paint(border, 0);
image = image.where(outside, 0).clip(border);



In [97]:
# Exporting path
path = image.getDownloadURL({
    'scale': 180,
    'crs': 'EPSG:4326'
    });

spatial_data_file = '2010.zip'
a = urllib.urlretrieve(path, spatial_data_file)
print a
zfile = zipfile.ZipFile(spatial_data_file)
zfile.extractall('LANDSAT_TOA/'+state_name)

('2010.zip', <httplib.HTTPMessage instance at 0x1051fc050>)


In [98]:
# Cleaning up the names of the files
channels = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6_VCID_2', 'B7']

for filename in os.listdir('LANDSAT_TOA/'+state_name):
    for channel in channels:
        try:
            prefix, band, extension = filename.partition(channel)
            if band==channel:
                os.rename(
                    'LANDSAT_TOA/'+state_name+'/'+filename,
                    'LANDSAT_TOA/'+state_name+'/'+state_name+'_'+year+'_'+band+extension)
        except:
            pass

In [99]:
for filename in os.listdir(folder_loc+'LANDSAT_TOA/'+state_name):
    print filename

Oregon_2010_B1.tfw
Oregon_2010_B1.tif
Oregon_2010_B2.tfw
Oregon_2010_B2.tif
Oregon_2010_B3.tfw
Oregon_2010_B3.tif
Oregon_2010_B4.tfw
Oregon_2010_B4.tif
Oregon_2010_B5.tfw
Oregon_2010_B5.tif
Oregon_2010_B6_VCID_2.tfw
Oregon_2010_B6_VCID_2.tif
Oregon_2010_B7.tfw
Oregon_2010_B7.tif
small
